In [ ]:
# loading items needed for Spark session
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net/c2d4u.tea

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-18 20:16:19--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  4.91MB/s    in 0.2s    

2022-09-18 20:16:20 (4.91 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [ ]:
# start a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# load Amazon review data into df
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Furniture_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3VR960AHLFKDV|          4|            0|          0|   N|                Y|
|R16LGVMFKIUT0G|          5|            0|          0|   N|                Y|
|R1AIMEEPYHMOE4|          5|            1|          1|   N|                Y|
|R1892CCSZWZ9SR|          3|            0|          0|   N|                Y|
|R285P679YWVKD1|          3|            0|          0|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# create df where total votes is equal or greater than 20
vine20 = vine_df.filter(vine_df.total_votes >= 20)
vine20.show(5) 

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|
| RR99CPG695T0I|          5|          215|        248|   N|                N|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# add helpful percentage column.  helpful % = helpful_votes / total_votes 
import pyspark.sql.functions as F
vine20_all = vine20.withColumn("helpful_p",(F.col("helpful_votes")/F.col("total_votes")))
vine20_all.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- helpful_p: double (nullable = true)



In [ ]:
vine20_all.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         helpful_p|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|0.9212121212121213|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|0.9130434782608695|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|               1.0|
| RR99CPG695T0I|          5|          215|        248|   N|                N|0.8669354838709677|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|0.9772727272727273|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
only showing top 5 rows



In [ ]:
# create new df where helpful percentage is greater than 50%
vine20_50 = vine20_all.filter(vine20_all.helpful_p >= 0.5)
vine20_50.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         helpful_p|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|0.9212121212121213|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|0.9130434782608695|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|               1.0|
| RR99CPG695T0I|          5|          215|        248|   N|                N|0.8669354838709677|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|0.9772727272727273|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
only showing top 5 rows



In [ ]:
vine20_all.columns

['review_id',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'helpful_p']

In [ ]:
# seperate out reviews where vine is Y (paid)
vine20_50Y = vine20_50.filter(vine20_50.vine == 'Y')
vine20_50Y.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         helpful_p|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2BQOD1R0228FN|          3|           17|         26|   Y|                N|0.6538461538461539|
| RC31RUPFOHBHQ|          4|          102|        117|   Y|                N|0.8717948717948718|
| REN3N1WITLF1Y|          5|           33|         37|   Y|                N|0.8918918918918919|
| R71RZQ9UZVG47|          4|           38|         47|   Y|                N|0.8085106382978723|
|R38NMQBH88HLM6|          4|           18|         24|   Y|                N|              0.75|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
only showing top 5 rows



In [ ]:
# total number of paid reviews (vine=Y)
vine20_50Y_C = vine20_50Y.count()
print(vine20_50Y_C)

136


In [ ]:
# Number of paid 5-star reviews (vine=Y)
vine20_50Y_5C = vine20_50Y.filter(vine20_50Y.star_rating == 5).count()
print(vine20_50Y_5C)

74


In [ ]:
# % of 5-start when reveiws are paid (vine=Y)
vine20_50Y_5p = vine20_50Y_5C/vine20_50Y_C
print(vine20_50Y_5p)

0.5441176470588235


In [ ]:
# seperate out reviews where vine is N (unpaid)
vine20_50N = vine20_50.filter(vine20_50.vine == 'N')
vine20_50N.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         helpful_p|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|0.9212121212121213|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|0.9130434782608695|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|               1.0|
| RR99CPG695T0I|          5|          215|        248|   N|                N|0.8669354838709677|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|0.9772727272727273|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
only showing top 5 rows



In [ ]:
# total number of unpaid reviews (vine=N)
vine20_50N_C = vine20_50N.count()
print(vine20_50N_C)

18019


In [ ]:
# Number of unpaid 5-star reviews (vine=N)
vine20_50N_5C = vine20_50N.filter(vine20_50N.star_rating == 5).count()
print(vine20_50N_5C)

8482


In [ ]:
# % of 5-start when reveiws are unpaid (vine=N)
vine20_50N_5p = vine20_50N_5C/vine20_50N_C
print(vine20_50N_5p)

0.47072534546867195
